In [1]:
from tensorflow.python.keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import random
import time

Using TensorFlow backend.


In [10]:
BATCH_SIZE = 256
SAMPLES_PER_EPOCH = 50000
EPOCH_NUM = 50

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b
  
def get_batch(features,labels,batch_size): 
    num_images = features.shape[0]
    idx = np.random.choice(num_images,
                           size=batch_size,
                           replace=False)
    features_batch = features[idx, :, :, :]
    labels_batch = labels[idx, :]
    return features_batch, labels_batch

In [4]:
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()

num_train, img_rows, img_cols, img_channels =  train_features.shape
num_test =  test_features.shape[0]
num_classes = np.unique(train_labels).shape[0]

train_features = train_features.astype('float32')/np.max(train_features)
test_features = test_features.astype('float32')/np.max(test_features)


train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(test_labels, num_classes)

print(train_features.shape, train_labels.shape)

(50000, 32, 32, 3) (50000, 10)


In [5]:
x = tf.placeholder(tf.float32 , shape=[None , img_rows,img_cols,img_channels]) 
y_ = tf.placeholder(tf.float32 , shape=[None, 10]) 
keep_prob = tf.placeholder(tf.float32)

test_features, test_labels = get_batch(test_features,test_labels,500)

In [7]:
with tf.name_scope('conv_1'): 
    conv1 = conv_layer(x , shape=[3, 3, img_channels, 64])
    conv1_pool = max_pool_2x2( conv1 )

with tf.name_scope('conv_2'): 
    conv2 = conv_layer(conv1_pool , shape=[3, 3, 64, 128])
    conv2_pool = max_pool_2x2 ( conv2 )

with tf.name_scope('conv_3'): 
    conv3 = conv_layer(conv2_pool , shape=[3, 3, 128, 256] )
    conv3_pool = max_pool_2x2 ( conv3 )
    conv3_flat = tf.contrib.layers.flatten(conv3_pool) 

with tf.name_scope('full_1'): 
    full_1 = tf.nn.relu(full_layer(conv3_flat , 512))

with tf.name_scope('dropout'): 
    full1_drop = tf.nn.dropout(full_1 , keep_prob=keep_prob) 

with tf.name_scope('activations'): 
    y_conv = full_layer(full1_drop , 10) 
    tf.summary.scalar('cross_entropy_loss',y_conv)	

with tf.name_scope('cross'): 
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv , labels=y_)) 

In [8]:
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv , 1), tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
print(test_features.shape, test_labels.shape)

(500, 32, 32, 3) (500, 10)


In [11]:
 with tf.device("/gpu:0"):   
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        start_time = time.time()
        
        for j in range(EPOCH_NUM):
            for i in range(SAMPLES_PER_EPOCH // BATCH_SIZE):
                batch_trainf, batch_trainl = get_batch(train_features,train_labels,BATCH_SIZE)
                sess.run(train_step , feed_dict={x: batch_trainf, y_: batch_trainl, keep_prob: 0.5}) 
            batch_trainf, batch_trainl = get_batch(test_features,test_labels,32)
            train_accuracy = sess.run(accuracy , feed_dict={x: batch_trainf, y_: batch_trainl, keep_prob: 1.0}) 
            print("time {}, epoch {}, training accuracy {}".format(time.time() - start_time, j, train_accuracy)) 

        test_accuracy = np.mean([sess.run(accuracy , feed_dict={x:test_features, y_:test_labels, keep_prob:1.0})])
        print("test accuracy: {}".format(test_accuracy))

time 11.170629978179932, epoch 0, training accuracy 0.5
time 22.2426176071167, epoch 1, training accuracy 0.53125
time 33.315584659576416, epoch 2, training accuracy 0.71875
time 44.401983976364136, epoch 3, training accuracy 0.625
time 55.49939465522766, epoch 4, training accuracy 0.625
time 66.6056764125824, epoch 5, training accuracy 0.65625
time 77.69407176971436, epoch 6, training accuracy 0.53125
time 88.78852772712708, epoch 7, training accuracy 0.8125
time 99.8799364566803, epoch 8, training accuracy 0.75
time 110.9821457862854, epoch 9, training accuracy 0.78125
time 122.07155275344849, epoch 10, training accuracy 0.75
time 133.16862177848816, epoch 11, training accuracy 0.53125
time 144.2646517753601, epoch 12, training accuracy 0.84375
time 155.369304895401, epoch 13, training accuracy 0.6875
time 166.47116947174072, epoch 14, training accuracy 0.6875
time 177.60183787345886, epoch 15, training accuracy 0.78125
time 188.7622423171997, epoch 16, training accuracy 0.8125
time 